In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import glob
import tqdm
import textgrids
import pandas as pd

from main import *

In [9]:
import tarfile
import shutil
import fsspec
from utils import generate_txt, get_subset_df, genorate_pps_df, make_tarfile, create_json_list

chunk = 2
generate_subset_tsv = True
pps_df_dir = '/home/knoriy/split_peoples_speech/pps_train.tsv'

root_path = '/home/knoriy/split_peoples_speech/'
dataset_name = 'test'

metadata_dir = "/mnt/knoriy/metadata.json"
tar_dir = "/mnt/knoriy/pps_train.tar"

# init Dirs
dataset_root_path = os.path.join(root_path, f'{dataset_name}')
dataset_textgrid_path = os.path.join(root_path, f'{dataset_name}_textgrids')
dataset_split_path = os.path.join(root_path, f'{dataset_name}_split')
s3 = fsspec.filesystem('s3')
s3_dest = f's-laion/peoples_speech/{dataset_name}_tars/'

In [3]:
if os.path.isfile(pps_df_dir):
    pps_df = pd.read_csv(pps_df_dir, sep='\t', header=None, names=['audio_filepath', 'text'])
else:
    pps_df = genorate_pps_df(metadata_dir)
    pps_df.to_csv(pps_df_dir, sep='\t', header=None, index=False)

In [4]:
pps_df

,audio_filepath,text
0,gov_DOT_uscourts_DOT_ca9_DOT_04-56618/gov_DOT_...,is there anything to wear it on this one or ha...
1,gov_DOT_uscourts_DOT_ca9_DOT_04-56618/gov_DOT_...,i thought the law was that we don't have juris...
2,gov_DOT_uscourts_DOT_ca9_DOT_04-56618/gov_DOT_...,review it yes you do yes you do
3,gov_DOT_uscourts_DOT_ca9_DOT_04-56618/gov_DOT_...,this you know what happened there was a single...
4,gov_DOT_uscourts_DOT_ca9_DOT_04-56618/gov_DOT_...,motion to abandon the quantity because nothing...
...,...,...
4320997,The_Capture_movie/Capture_DOT_mp3_00011.flac,reward of all our marriage
4320998,The_Capture_movie/Capture_DOT_mp3_00012.flac,so i traveled back the way i'd come on it this...
4320999,The_Capture_movie/Capture_DOT_mp3_00013.flac,you state your business mister
4321000,The_Capture_movie/Capture_DOT_mp3_00014.flac,come out yet i don't want to be just that


In [5]:
src_file_obj = tarfile.open(tar_dir, mode='r')
print('opening file: This may take some time\n')

file_names_full_list = src_file_obj.getnames()
file_names_full_list = [i for i in file_names_full_list if '.flac' in i]

opening file: This may take some time



In [10]:
for i in tqdm.tqdm(range(0, len(file_names_full_list), chunk), desc='Chunks remaining: '):
    for file_name in tqdm.tqdm(file_names_full_list[i:i + chunk], desc="Extracting Files: "):
        src_file_obj.extract(file_name, f'./{dataset_name}/')

    if generate_subset_tsv == True:
        df = get_subset_df(f'{dataset_root_path}/**/*.flac', pps_df)

    save_all_text_to_file(df, dataset_name)

    # Convert Flac to wav
    convert_all_to_wav(df, os.path.join(root_path, dataset_name))

    # Get audio text alignments and split audio
    generate_textgrids(os.path.join(root_path, dataset_name))
    split_all_audio_files(dataset_textgrid_path, dataset_root_path)

    # # Upload Split files to s3
    tar_file_path = make_tarfile(f'{dataset_split_path}', f'{dataset_root_path}/{i}.tar')
    # s3.put(tar_file_path, os.path.join(s3_dest, os.path.basename(tar_file_path)))
    # print('File Uploaded to: ', os.path.join(s3_dest, os.path.basename(tar_file_path)))

    # # Upload sizes.jsonl to s3
    # create_json_list(json_sizes_path, {'filename':tar_file_path, 'num_samples':len(df)})
    # s3.put(json_sizes_path, 's-laion/peoples_speech/sizes.jsonl')

    # shutil.rmtree(dataset_root_path)
    # shutil.rmtree(dataset_textgrid_path)
    # shutil.rmtree(dataset_split_path)
    break

Converting .flac files to .wav: 100%|██████████| 2/2 [00:00<00:00,  3.94it/s]


INFO - Setting up corpus information...
INFO - Loading corpus from source files...


  0%|          | 0/2 [00:00<?, ?it/s]

INFO - Found 1 speaker across 2 files, average number of utterances per speaker: 2.0
INFO - Initializing multiprocessing jobs...
WARNING - Number of jobs was specified as 3, but due to only having 1 speakers, MFA will only use 1 jobs.
INFO - Creating corpus split for feature generation...
INFO - Generating base features (mfcc)...
INFO - Generating MFCCs...


  0%|          | 0/2 [00:00<?, ?it/s]

...

100%|██████████| 2/2 [00:01<00:00,  1.40it/s]


INFO - Exporting TextGrids to /home/knoriy/split_peoples_speech/test_textgrids...
INFO - Collecting phone and word alignments from alignment lattices...


100%|██████████| 2/2 [00:01<00:00,  1.77it/s]


INFO - Finished exporting TextGrids to /home/knoriy/split_peoples_speech/test_textgrids!
INFO - Done! Everything took 32.29901075363159 seconds


Chunks remaining:   0%|          | 0/2160501 [00:37<?, ?it/s]


In [11]:
src_file_obj.close()

# AWS S3

In [ ]:
%%bash

nohup sh -c "wget https://the-peoples-speech-west-europe.bj.bcebos.com/part-00000-07a8f0d3-6d27-4299-887a-dc12a6d72f8d-c000.tar?authorization=bce-auth-v1/0ef6765c1e494918bc0d4c3ca3e5c6d1/2021-12-03T06%3A30%3A22Z/-1/host/444b9c082ceffd10f38bb965679ed9ec12202836831e111dd193fde281062d26 -O - | aws s3 cp - s3://s-laion/peoples_speech/train_clean_pps.tar" &;
nohup sh -c "wget https://the-peoples-speech-west-europe.bj.bcebos.com/part-00000-4e132642-c01c-4db6-9db0-a1e19193f6f8-c000.json?authorization=bce-auth-v1/0ef6765c1e494918bc0d4c3ca3e5c6d1/2021-12-03T06%3A31%3A22Z/-1/host/d7dacf3c31d2e3670d82727636ce234be27a9128df7a80883b84b4a3d8c7f6c0 -O - | aws s3 cp - s3://s-laion/peoples_speech/Manifest.json" &;


## Make Tar

In [1]:
import glob
import random
import json

def tardir(file_path, tar_name, n_entry_each, audio_ext=".flac", text_ext=".json", shuffle=True, start_idx=0, delete_file=False):
    """
    This function create the tars that includes the audio and text files in the same folder
    @param file_path      | string  | the path where audio and text files located
    @param tar_name       | string  | the tar name
    @param n_entry_each   | int     | how many pairs of (audio, text) will be in a tar
    @param audio_ext      | string  | the extension of the audio
    @param text_ext       | string  | the extension of the text
    @param shuffle        | boolean | True to shuffle the file sequence before packing up
    @param start_idx      | int     | the start index of the tar
    @param delete_file    | boolean | True to delete the audio and text files after packing up
    """
    filelist = glob(file_path+'/*'+audio_ext)

    if shuffle:
        random.shuffle(filelist)
    count = 0
    n_split = len(filelist) // n_entry_each
    if n_split * n_entry_each != len(filelist):
        n_split += 1
    size_dict = {
        os.path.basename(tar_name) + str(i) + ".tar": n_entry_each
        for i in range(n_split)
    }
    if n_split * n_entry_each != len(filelist):
        size_dict[os.path.basename(tar_name) + str(n_split - 1) + ".tar"] = (
            len(filelist) - (n_split - 1) * n_entry_each
        )
    for i in tqdm(range(start_idx, n_split + start_idx)):
        with tarfile.open(tar_name + str(i) + ".tar", "w") as tar_handle:
            for j in range(count, len(filelist)):
                audio = filelist[j]
                basename = ".".join(audio.split(".")[:-1])
                text_file_path = os.path.join(file_path, basename + text_ext)
                audio_file_path = os.path.join(file_path, audio)
                tar_handle.add(audio_file_path)
                tar_handle.add(text_file_path)
                if delete_file:
                    os.remove(audio_file_path)
                    os.remove(text_file_path)
                if (j + 1) % n_entry_each == 0:
                    count = j + 1
                    break
        tar_handle.close()
    # Serializing json
    json_object = json.dumps(size_dict, indent=4)
    # Writing to sample.json
    with open(os.path.join(os.path.dirname(tar_name), "sizes.json"), "w") as outfile:
        outfile.write(json_object)
    return size_dict